In [275]:
import numpy as np
import csv
import random
import copy

In [276]:
class gene():
    def __init__(self,name, data):
        self.name = name
        self.data = data
    def __repr__(self):
        return f"point:{self.name}"
    
    def __eq__(self,other):
        if self.name == other.name:
            return True
        else:
            return False

In [277]:
class Chromosome():
    def __init__(self, list_genes):
        self.fitness = 0 
        self.cost = 0 
        self.list_genes = list_genes 
    def __repr__(self):
        return f'{self.list_genes} cost: {self.cost} fitness:{self.fitness}'
        

In [278]:
def create_data(path):
    lst = []
    with open(path, 'r') as f:
            lines = list(csv.DictReader(f))
    for line in lines:
            lst.append(gene(line['City'], [line['x'], line ['y']]))
    return lst

In [279]:
def distance(povit, point):
    povit = np.array(povit, dtype= float)
    point = np.array(point, dtype= float)
    dist =np.sum((point - povit)**2)**0.5
    return dist

In [280]:
def Matrix_distance(intinal_chromosome):
    distance_matrix = np.zeros((len(intinal_chromosome), len(intinal_chromosome)))
    for i in range(len(intinal_chromosome)):
        for j in range(len(intinal_chromosome)):
            if i != j:
                distance_matrix[i,j] = distance(intinal_chromosome[i].data,intinal_chromosome[j].data)
    return distance_matrix

In [281]:
def cost_chromosome(chromosome, distance_matrix):
    total_distance = 0
    for i in range(len(chromosome.list_genes)-1):
        city1 = chromosome.list_genes[i]
        city2 = chromosome.list_genes[i+1]
        total_distance += distance_matrix[int(city1.name)-1, int(city2.name)-1]
    return total_distance +distance(chromosome.list_genes[-1].data,chromosome.list_genes[0].data)

In [282]:
def create_population(num,distance_matrix):
    lst = create_data('15-Points.csv')
    population = []
    for i in range(num):
        c = Chromosome(list(np.random.permutation(lst)))
        # return to the start city again
        c.cost = cost_chromosome(c, distance_matrix)
        c.fitness = 1/c.cost 
        population.append(c)

    return population

In [283]:
def Fitness(chromosome, distance_matrix):
    return 1/cost_chromosome(chromosome, distance_matrix)

In [284]:
def sort_fitnesses(population):
    population =sorted(population,key =lambda x:x.fitness, reverse= True)
    return population

In [285]:
def random_chromosomes(population, num):
    return random.choices(population, k=num)

In [286]:
def selection(chromosomes):
    chromosomes = sorted(chromosomes, key= lambda x:x.fitness, reverse= True)
    return chromosomes[0]

In [287]:
def PMC_crossover(parent1, parent2,distance_matrix):
    crossover_point = random.choices(range(len(parent1.list_genes)), k=2)
    child1 = Chromosome(copy.deepcopy(parent1.list_genes))
    child2 = Chromosome(copy.deepcopy(parent2.list_genes))
    #print(child1)
    for i in range(min(crossover_point), max(crossover_point)):
        temp1 = parent2.list_genes[i]
        temp2 = parent1.list_genes[i]
      
        indx1 = child1.list_genes.index(temp1)
        child1.list_genes[indx1] = child1.list_genes[i]
   
        indx2 = child2.list_genes.index(temp2)
        child2.list_genes[indx2] = child2.list_genes[i]
        
        child1.list_genes[i] = temp1
        child2.list_genes[i] = temp2
       
    child1.cost = cost_chromosome(child1, distance_matrix)
    child2.cost = cost_chromosome(child2, distance_matrix)

    child1.fitness = 1/child1.cost
    child2.fitness = 1/child2.cost
    #print(child1)
    return child1, child2

In [288]:
def crossover(probability, oldpop, newpop,distance_matrix):
    num = int(len(oldpop)*probability)
    while True:
        some_chromosomes  = random_chromosomes(oldpop, num)
        parent1 = selection(some_chromosomes)
        #to certain that parent2 not the same parent1
        some_chromosomes2  = random_chromosomes(oldpop, num)
        parent2 = selection(some_chromosomes2)
      
        child1,child2 = PMC_crossover(parent1, parent2,distance_matrix)
        #print(child1)
     
        newpop.append(child1)
        newpop.append(child2)
       
        if len(newpop) == len(oldpop):
            return newpop

In [289]:
def mutation(probability,newpop, num_elite,distance_matrix):
    num = int(len(newpop)*probability)
    for mut in range(num):
        ch=  random.choices(range(num_elite,len(newpop)))[0]
        j =  random.choices(range(len(newpop[0].list_genes)), k=2)
        max_i = max(j)
        for i in range(min(j),  max(j)):
                newpop[ch].list_genes[max_i],  newpop[ch].list_genes[i] =  newpop[ch].list_genes[i], newpop[ch].list_genes[max_i]
                max_i-=1
        
        newpop[ch].cost = cost_chromosome(newpop[ch], distance_matrix)
        newpop[ch].fitness = 1/newpop[ch].cost
       
    return newpop

In [290]:
def Gene_Algo(oldpop, num_elite, probility_mution, probability_crossover, distance_matrix):
        newpop = []
 
        elites = sort_fitnesses(oldpop)
        newpop.extend(elites[:num_elite])

        newpop = crossover(probability_crossover, oldpop, newpop,distance_matrix)
        #newpop = mutation(probility_mution,newpop, num_elite,distance_matrix)
        newpop =  mutation(probility_mution, newpop, num_elite,distance_matrix)
       
        return newpop

In [291]:
if __name__ == "__main__":
    probility_mution = 0.6
    probability_crossover = 0.8
    num_pop = 400
    num_elite = 4
    num_gene = 100
    lst = create_data('15-Points.csv')
    distance_matrix = Matrix_distance(lst)
    population = create_population(num_pop,distance_matrix)
    for i in range (num_gene):   
              population = Gene_Algo(population, num_elite, probility_mution, probability_crossover, distance_matrix)
              population = sort_fitnesses(population)
              #print(population[0])
              #print(cost_chromosome(population[0], distance_matrix))
    result = sort_fitnesses(population)[0]
    print(cost_chromosome(result, distance_matrix))
    result.list_genes.append(result.list_genes[0])
    print(result)
    

284.3810904080332
[point:13, point:2, point:15, point:9, point:5, point:7, point:3, point:12, point:14, point:10, point:8, point:6, point:4, point:11, point:1, point:13] cost: 284.3810904080332 fitness:0.003516408206203826


In [292]:
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
%matplotlib qt

In [293]:
def plot_sol(solution):
    x = []
    y = []
    names=[]
    for gene in solution.list_genes:
        x.append(float(gene.data[0]))
        y.append(float(gene.data[1]))
        names.append('point'+gene.name)
    fig, ax = plt.subplots()
    def animate(i):
        ax.clear()
        ax.plot(x[:i+1], y[:i+1], marker = 'o')
    plt.plot(x, y, marker = 'o')
    for i in range(len(x)):
        plt.text(x[i]+0.1, y[i]+0.1, names[i], fontsize=10)
    plt.title(f'Genetic Algorithm Cost {result.cost}')
    plt.show()

In [294]:
plot_sol(result)

In [295]:
x = []
y = []
names = []
for gene in result.list_genes:
        x.append(float(gene.data[0]))
        y.append(float(gene.data[1]))
        names.append('point'+gene.name)

fig, ax = plt.subplots()

def animate(i):
    ax.clear()
    ax.plot(x[:i+1], y[:i+1], marker = 'o')
    plt.text(x[i]+0.1, y[i]+0.1, names[i], fontsize=10)
    plt.title(f'Genetic Algorithm Cost {result.cost}')

    plt.ylim(-50, 30)
    plt.xlim(-90, 10)

ani = FuncAnimation(fig, animate, frames=20, interval=300, repeat=False)
plt.show()